In [1]:
import os
import osmnx as ox
import networkx as nx
import folium
import matplotlib.pyplot as plt

%matplotlib inline

# Output directory
output_dir = "maps_week1"
os.makedirs(output_dir, exist_ok=True)

print("Environment ready")


Environment ready


In [2]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="AI_Green_Transportation_APP")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=3)

CITY_NAME = "Bengaluru, India"

# User input - can be ANY location
origin_input = input("Enter Origin location (anywhere in Bengaluru): ")
dest_input = input("Enter Destination location (anywhere in Bengaluru): ")

# Geocode user inputs
origin = geocode(origin_input + ", Bengaluru, India", timeout=10)
destination = geocode(dest_input + ", Bengaluru, India", timeout=10)

# Handle invalid inputs
if origin is None:
    raise ValueError(f"Origin not found: {origin_input}")

if destination is None:
    raise ValueError(f"Destination not found: {dest_input}")

origin_name = origin.address
dest_name = destination.address

origin_lat, origin_lon = origin.latitude, origin.longitude
dest_lat, dest_lon = destination.latitude, destination.longitude

print("Origin:", origin_name)
print("Coordinates:", (origin_lat, origin_lon))
print("Destination:", dest_name)
print("Coordinates:", (dest_lat, dest_lon))


Enter Origin location (anywhere in Bengaluru):  Koramangala Bengaluru
Enter Destination location (anywhere in Bengaluru):  Majestic Bengaluru


Origin: Thoughtworks Technologies (India) Pvt. Ltd., 147/F, ACR Mansion, 8th Main Road, Jakkasandra Block, Koramangala, Bengaluru South City Corporation, Bengaluru, Bangalore South, Bengaluru Urban, Karnataka, 560034, India
Coordinates: (12.9287596, 77.6287122)
Destination: State Bank of India, Nelamangala - Majestic Service Road, Mahalakshmi Layout, Bengaluru West City Corporation, Bengaluru, Bangalore North, Bengaluru Urban, Karnataka, 560022, India
Coordinates: (13.0237386, 77.5481786)


In [3]:
print("Downloading road network for", CITY_NAME)

G = ox.graph_from_place(CITY_NAME, network_type="drive")

print("Graph ready with", len(G.nodes), "nodes &", len(G.edges), "edges.")


Graph ready with 154471 nodes & 392383 edges.


In [4]:
origin_node = ox.nearest_nodes(G, origin_lon, origin_lat)
destination_node = ox.nearest_nodes(G, dest_lon, dest_lat)

shortest_route = nx.shortest_path(G, origin_node, destination_node, weight='length')
shortest_distance_m = nx.shortest_path_length(G, origin_node, destination_node, weight='length')
shortest_distance_km = shortest_distance_m / 1000

print("Shortest Distance:", round(shortest_distance_km, 2), "km")
print("Route Points:", len(shortest_route))


Shortest Distance: 17.28 km
Route Points: 232


In [5]:
# Convert route to a list of lat/lon coordinates
route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_route]

# Create a folium map centered on the first route point
route_map = folium.Map(location=route_coords[0], zoom_start=12)

# Add route polyline to the map
folium.PolyLine(route_coords, color="blue", weight=5).add_to(route_map)

# Save map as HTML
map_path = os.path.join(output_dir, "week1_shortest_route_map_bangalore.html")
route_map.save(map_path)

print("Week 1 map saved at:", os.path.abspath(map_path))


Week 1 map saved at: C:\AI_Green_Transportation\maps_week1\week1_shortest_route_map_bangalore.html
